In [1]:
import os
import numpy as np
import pydub
import datetime
import time
import re

workfolder = '/Users/dingran/projects/speech_transciption'

In [2]:
# audio_file = 'BKorP55Aqvg/BKorP55Aqvg.wav'
# youtube_sub = 'BKorP55Aqvg/BKorP55Aqvg.en-GB.vtt'
# lang = 'en_GB'
# filename_template = os.path.join(workfolder, 'BKorP55Aqvg/segment_{}.{}')

# audio_file = '7W_qrc-TkR8/7W_qrc-TkR8.wav'
# youtube_sub = '7W_qrc-TkR8/7W_qrc-TkR8.en.vtt'
# lang = 'en_US'
# filename_template = os.path.join(workfolder, '7W_qrc-TkR8/segment_{}.{}')

# audio_file = 'HTgYHHKs0Zw/HTgYHHKs0Zw.wav'
# youtube_sub = 'HTgYHHKs0Zw/HTgYHHKs0Zw.en.vtt'
# lang = 'en_US'
# filename_template = os.path.join(workfolder, 'HTgYHHKs0Zw/segment_{}.{}')


audio_file = 'PdUKP4fFiBY/PdUKP4fFiBY.wav'
youtube_sub = 'PdUKP4fFiBY/PdUKP4fFiBY.en.vtt'
lang = 'en_US'
filename_template = os.path.join(workfolder, 'PdUKP4fFiBY/segment_{}.{}')

with open(youtube_sub, 'r') as f:
    s =f.readlines()
    s=[x.strip() for x in s]
    s = [x for x in s if x!='']

In [3]:
a = re.compile(r'(\d+):(\d+):(\d+).(\d+)')

In [4]:
transcript_list = []
for l in s:
    timestamp = a.findall(l)
    if timestamp and '-->' in l:
        start_time = int(timestamp[0][0])*3600+int(timestamp[0][1])*60+int(timestamp[0][2])+int(timestamp[0][3])*1e-3
        stop_time = int(timestamp[1][0])*3600+int(timestamp[1][1])*60+int(timestamp[1][2])+int(timestamp[1][3])*1e-3
        transcript_list.append([start_time, stop_time, []])
    else:
        if transcript_list:
            transcript_list[-1][-1] = l

In [5]:
transcript_list_merged = []
acumu_time = 0
text = ''
for t in transcript_list:
    if acumu_time ==0:
        t_start = t[0]
    acumu_time = t[1]-t[0] + acumu_time
    text += ' '+t[2]
    if acumu_time > 5:
        transcript_list_merged.append([t_start, t[1], text])
        acumu_time = 0
        text = ''

print transcript_list_merged
        

[[0.0, 7.508, " So, if you can just use one of these. Whoa, you've got a huge dongle. Thanks. It gets the job done, I guess."], [7.508, 14.148, " I've seen bigger. I have a really small one. Well, mine's normal size, but it doesn't work like it used to."], [14.148, 20.979, ' Hey guys, what is going on in there? Presentation. Paul tried to get his up. Tripp, I think yours is too wide.'], [20.979, 26.06, " I've got two. Can you use them at the same time? Hmm, never tried it. Has anyone seem mine?"], [26.06, 32.259, " It's resting on top of the sac. Tripp, is your more wider than it is long? I've never seen that before."], [32.259, 37.838, " Wait, Tara, do you have one? Yeah, but it's at home in my drawer. You know, mine was like this once. Mine bends to the left."], [37.838, 43.934, " Whoa, I think you're stepping on mine. Is your purple? Just the top part. Actually, Clint, let me hold yours for a second."], [43.934, 48.98, " Sure. Why is this so hard? Did you try taking it out and putti

In [6]:
from pydub import AudioSegment
sound = AudioSegment.from_wav(audio_file)


In [7]:

i=0
for t in transcript_list_merged:
    segment = sound[t[0]*1000:t[1]*1000]
    segment = segment.set_channels(1)
    segment.export(filename_template.format(i, 'wav'), format='wav')
    with open(filename_template.format(i, 'txt'), "w") as text_file:
        text_file.write(t[2])
    i+=1
N= i-1

In [8]:
import io
from pydub.utils import mediainfo
def transcribe_file(speech_file):
    """Transcribe the given audio file."""
    from google.cloud import speech
    speech_client = speech.Client()
    
    info = mediainfo(speech_file)
    #print info

    with io.open(speech_file, 'rb') as audio_file:
        content = audio_file.read()
        audio_sample = speech_client.sample(
            content=content,
            source_uri=None,
            encoding='LINEAR16',
            sample_rate_hertz=int(info['sample_rate']))

    try:
        alternatives = audio_sample.recognize(lang)
#         for alternative in alternatives:
#             print('Transcript: {}'.format(alternative.transcript))

        return alternatives[0].transcript
    except:
        return ''

In [9]:
for i in range(N):
    google_trans = transcribe_file(filename_template.format(i, 'wav'))
    print 'Google:\n', google_trans
    with open(filename_template.format(i, 'ggl.txt'), "w") as text_file:
        text_file.write(google_trans)
    print 'Official:\n', transcript_list_merged[i][2]

Google:
puppies
Official:
 So, if you can just use one of these. Whoa, you've got a huge dongle. Thanks. It gets the job done, I guess.
Google:
I've seen bigger
Official:
 I've seen bigger. I have a really small one. Well, mine's normal size, but it doesn't work like it used to.
Google:
what is going on in their presentation trip I think you're just too why
Official:
 Hey guys, what is going on in there? Presentation. Paul tried to get his up. Tripp, I think yours is too wide.
Google:
I've got to commune with the same time never tried it
Official:
 I've got two. Can you use them at the same time? Hmm, never tried it. Has anyone seem mine?
Google:
resting on top of the sack
Official:
 It's resting on top of the sac. Tripp, is your more wider than it is long? I've never seen that before.
Google:
yeah but it's at home like this one to the left
Official:
 Wait, Tara, do you have one? Yeah, but it's at home in my drawer. You know, mine was like this once. Mine bends to the left.
Google:
I t